In [ ]:
import osmnx as ox
import networkx as nx
from multiprocessing import Pool
import itertools


import matplotlib.pyplot as plt


In [ ]:
try:
    import importlib
    import src.graph_edit_functions
    import src.centrality_algorithms
    import src.plot_functions
    importlib.reload(src.graph_edit_functions)
    importlib.reload(src.plot_functions)
    importlib.reload(src.centrality_algorithms)
except Exception as e:
    print(e)

from src.graph_edit_functions import *
from src.centrality_algorithms import *
from src.plot_functions import *

In [ ]:
g = ox.graph_from_place("Wrocław, Poland", network_type="drive")

In [ ]:


g = get_sc(g)


In [ ]:
g.number_of_nodes()

In [ ]:
from typing import (
    List,
    Union,
    Dict,
)
import numpy as np

asc_highway_weights: Dict[str, float] = {
    'residential': 2,
    'tertiary': 2,
    'living_street': 4,
    'secondary': 6,
    'unclassified': 1,
    'primary': 8,
    'secondary_link': 5,
    'tertiary_link': 5,
    'primary_link': 5,
    'motorway_link': 5,
    'motorway': 10,
    'trunk_link': 5,
    'trunk': 10,
}

desc_highway_weights: Dict[str, float] = {
    'residential': 10,
    'tertiary': 10,
    'living_street': 8,
    'secondary': 6,
    'unclassified': 11,
    'primary': 10,
    'secondary_link': 7,
    'tertiary_link': 7,
    'primary_link': 7,
    'motorway_link': 7,
    'motorway': 2,
    'trunk_link': 7,
    'trunk': 2,
}

def multi_road_type_weight(multi_road_type: List[str], weight_dict: Dict[str, float]):
    return np.mean(list(weight_dict[road_type] for road_type in multi_road_type))

def road_type_weight(road_type: Union[str, List[str]], weight_dict: Dict[str, float]):
    return weight_dict[road_type] if type(road_type) == type("") else multi_road_type_weight(road_type, weight_dict)



In [ ]:
METRIC_WEIGHTS = {
    "bc": desc_highway_weights,
    "pr": asc_highway_weights,
    "cc": asc_highway_weights,
    "ec": asc_highway_weights,
    "ecc": desc_highway_weights
}

def assign_weights(graph):
    for metric in METRIC_WEIGHTS:
        nodes, edges = ox.graph_to_gdfs(g)
        edge_weights = edges.highway.map(lambda road_type: road_type_weight(road_type, METRIC_WEIGHTS[metric]))
        nx.set_edge_attributes(g, edge_weights, f'{metric}_weight')

assign_weights(g)

In [ ]:
ox.plot_graph(g)

## Betweenness Centrality

In [ ]:
bc_dict = betweenness_centrality_parallel(g, processes=8)

In [ ]:

basic_plot(g, bc_dict, title="betweenness_centrality", path="images/wrocław/wrc-betweenness-centrality.png")

## Weighted Betweenness Centrality

In [ ]:
wbc_dict = betweenness_centrality_parallel(g, processes=8, weight="bc_weight")

In [ ]:
basic_plot(g, wbc_dict, title="weighted_betweenness_centrality", path="images/wrocław/wrc-weighted-betweenness-centrality.png")

 ## Pagerank

In [ ]:
pr_dict = nx.pagerank(g)

In [ ]:
basic_plot(g, pr_dict, title="pagerank", path="images/wrocław/wrc-pagerank-centrality.png")

## Weighted Pagerank

In [ ]:
wpr_dict = nx.pagerank(g, weight = "pr_weight")

In [ ]:
basic_plot(g, wpr_dict, title="weighted_pagerank", path="images/wrocław/wrc-weighted-pagerank-centrality.png")

## Clustering Coefficient

In [ ]:
# create weighted graph from g
g_not_multi = ox.get_digraph(g)

c_dict = nx.clustering(g_not_multi)

In [ ]:
basic_plot(g, c_dict, title="cc", path="images/wrocław/wrc-clustering-centrality.png")

## Weighted Clustering Coefficient

In [ ]:
# create weighted graph from g
g_not_multi = ox.get_digraph(g)

wc_dict = nx.clustering(g_not_multi, weight="cc_weigth")

nx.set_node_attributes(g, values=wc_dict, name="w_clustering_coefficient")

In [ ]:
basic_plot(g, wc_dict, title="cc", path="images/wrocław/wrc-weighted-clustering-centrality.png")

## Eigenvector centrality

In [ ]:
ec_dict = nx.eigenvector_centrality_numpy(g)


In [ ]:
basic_plot(g, ec_dict, title="eigen", path="images/wrocław/wrc-eigenvector-centraility.png")

In [ ]:
aec_dict = nx.pagerank(g, alpha=0.05)


In [ ]:
basic_plot(g, aec_dict, title="almost_eigen", path="images/wrocław/wrc-almost-eigenvector-centraility.png")

## Weighted Eigenvector Centrality

In [ ]:
wec_dict = nx.eigenvector_centrality_numpy(g, weight = 'ec_weight')


In [ ]:
basic_plot(g, wec_dict, "weighted_eigenvector_centraility", path="images/wrocław/wrc-weighted-eigenvector-centrality.png")

In [ ]:
waec_dict = nx.pagerank(g,weight="ec_weight", alpha=0.05, max_iter = 1000)


## Eccentricity

In [ ]:
ecc_dict = nx.eccentricity(g)


In [ ]:
basic_plot(g, ecc_dict, "eccentricity",path="images/wrocław/wrc-eccentiricity.png")

## Weighted Eccentricity

In [ ]:
wecc_dict = weighted_eccentricity(g, "ecc_weight")

In [ ]:
basic_plot(g, wecc_dict, "weighted_eccentricity", path="images/wrocław/wrc-weighted-eccentricity.png")

## Random Walk Betweenness